# **Importações e uploads necessários**

In [ ]:
import spacy
import re
import glob
#import sys
import json
import pandas as pd
from collections import defaultdict

In [ ]:
!python -m spacy download pt_core_news_lg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 568.2/568.2 MB 2.1 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('pt_core_news_lg')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
df = pd.read_json('/content/drive/MyDrive/final_dataframe.json')
df_by_id_chat = df.set_index('_id_chat')
df_by_id_chat

In [ ]:
id_dialogos = df['_id_chat'].unique()
id_dialogos.size

1388

# **Funções extras (validação do CPF)**

In [ ]:
# validação do cpf

def valida_cpf(cpf_string):

    # retira apenas os dígitos do cpf

    numeros = [int(digito) for digito in cpf_string if digito.isdigit()]

    quant_digitos = False
    validacao1 = False
    validacao2 = False

    if len(numeros) == 11:
        quant_digitos = True

        # validação do primeiro dígito

        soma_produtos = sum(a*b for a, b in zip (numeros[0:9], range (10, 1, -1)))
        digito_esperado = (soma_produtos * 10 % 11) % 10
        if numeros[9] == digito_esperado:
            validacao1 = True

        # validação do segundo dígito

        soma_produtos1 = sum(a*b for a, b in zip(numeros [0:10], range (11, 1, -1)))
        digito_esperado1 = (soma_produtos1 *10 % 11) % 10
        if numeros[10] == digito_esperado1:
            validacao2 = True

        # validação geral

        if quant_digitos == True and validacao1 == True and validacao2 == True:
            return True
        else:
            return False

    else:
        return False



# **Extração dos predicados do usuário através das entidades**

In [ ]:
def entities_extraction(phrase):

    nlp = spacy.load("pt_core_news_lg")

    # criação da regra da entidade e adição ao pipeline

    ruler = nlp.add_pipe("entity_ruler", before='ner')

    ruler.phrase_matcher = spacy.matcher.PhraseMatcher(nlp.vocab, attr="SHAPE")

    # entidades e seus respectivos padrões

    patterns = [
                    {
                        "label": "DOENCA", "pattern": [ {"LOWER": "coronavírus"} ], "id": "doenca"
                    },
                    {
                        "label": "DOENCA", "pattern": [ {"LOWER": "covid-19"} ], "id": "doenca"
                    },
                    {
                        "label": "ORG", "pattern": "Governo do Estado", "id": "organizacao"
                    },
                    {
                        "label": "TELEFONE", "pattern": [ {"TEXT": {"REGEX": "^\(?\d{2}\)?[-.\s]?\d{5}[-.\s]?\d{4}$"}} ], "id": "telefone"
                    },
                    {
                        "label": "TELEFONE", "pattern": '(49) 30492-2949', "id": "telefone"
                    },
                    {
                        "label": "CPF", "pattern": [ {"TEXT": {"REGEX": "(^(\d){11}$)"}} ], "id": "cpf"
                    },
                    {
                        "label": "CEP", "pattern": [ {"TEXT": {"REGEX": "(^(\d){8}$)"}} ], "id": "cep"
                    },
                    {
                        "label": "CEP", "pattern": [ {"TEXT": {"REGEX": "(^(\d){5}-(\d){3}$)"}} ], "id": "cep"
                    },
                    {
                        "label": "GENERO", "pattern": [ {"LOWER": "homem"} ], "id": "genero"
                    },
                    {
                        "label": "GENERO", "pattern": [ {"LOWER": "mulher"} ], "id": "genero"
                    },
                    {
                        "label": "DATA_NASC", "pattern": [ {"TEXT": {"REGEX": "^(0[1-9]|[12][0-9]|3[01])\/(0[1-9]|1[1,2])\/(19|20)\d{2}$"}} ], "id": "data_nascimento"
                    }
              ]

    # adição dos padrões as regras

    ruler.add_patterns(patterns)


    # criação do documento a partir do texto

    doc = nlp(phrase)

    # extração das entidades e suas respectivas labels

    find_labels = []

    for ent in doc.ents:
      if (ent.label_ != "MISC" and ent.label_ != "ORG" and ent.label_ != "DOENCA" and ent.text != "CPF"):
        if (ent.label_ == "PER"):
          ent.label_ = "NOME"
        elif (ent.label_ == "CPF"):
          if not valida_cpf(ent.text):
            ent.label_ = "TELEFONE"
        print (ent.text, ent.label_)
        find_labels.append(ent.label_)


    return find_labels


In [ ]:
def predicates():

    intentions_predicates = {'send-info-name': False, 'send-info-cpf': False, 'send-info-birthday': False, 'send-info-location': False,
                            'send-info-postal-code': False, 'send-info-gender': False, 'send-info-phone-number': False}
    entities_predicates = {'have-info-name': False, 'have-info-cpf': False, 'have-info-birthday': False, 'have-info-location': False,
                          'have-info-postal-code': False, 'have-info-gender': False, 'have-info-phone-number': False }

    return intentions_predicates, entities_predicates

In [ ]:
def predicates_extraction(user_phrase, int_predicates, ent_predicates):

    phrase = user_phrase

    list_predicates = entities_extraction(phrase)

    intentions_predicates = int_predicates
    entities_predicates = ent_predicates
    user_predicates = {}

    for i in list_predicates:
      if (i == 'NOME'):
        intentions_predicates['send-info-name'] = True
        entities_predicates['have-info-name'] = True
      elif (i == 'CPF'):
        intentions_predicates['send-info-cpf'] = True
        entities_predicates['have-info-cpf'] = True
      elif (i == 'DATA_NASC'):
        intentions_predicates['send-info-birthday'] = True
        entities_predicates['have-info-birthday'] = True
      elif (i == 'LOC'):
        intentions_predicates['send-info-location'] = True
        entities_predicates['have-info-location'] = True
      elif (i == 'CEP'):
        intentions_predicates['send-info-postal-code'] = True
        entities_predicates['have-info-postal-code'] = True
      elif (i == 'GENERO'):
        intentions_predicates['send-info-gender'] = True
        entities_predicates['have-info-gender'] = True
      elif (i == 'TELEFONE'):
        intentions_predicates['send-info-phone-number'] = True
        entities_predicates['have-info-phone-number'] = True

    user_predicates.update(intentions_predicates)
    user_predicates.update(entities_predicates)

    return intentions_predicates, entities_predicates, user_predicates

# **Extração das ações através das respostas do chatbot**

In [ ]:
def find_actions(chatbot_phrase):

    phrase = chatbot_phrase

    actions_map = {
        'error-treatment': 'Não conseguir entender o que você quis dizer',
        'error-treatment-voice-message': 'Desculpe, ainda não consigo entender conversas por voz.',
        'show-welcome-message': 'Olá! Bem-vindo(a) ao serviço de atendimento online da Secretaria da Saúde do Estado do Ceará!',
        'start-dialog': 'Do que você precisa agora? (Digite o número da opção desejada) 1.  Atendimento online sobre o novo Coronavírus (COVID-19)  2.  Informações sobre o novo Coronavírus (COVID-19)  3.  Cuidados com a Saúde Mental 4. Outros',
        'error-treatment-start-dialog': 'Desculpe, não entendi sua dúvida. Do que você precisa agora? (Digite o número da opção desejada)   1.  Atendimento online sobre o novo Coronavírus (COVID-19)  2.  Informações sobre o novo Coronavírus (COVID-19)  3.  Cuidados com a Saúde Mental 4. Outros',
        'start-online-service-v1': 'As informações compartilhadas aqui são protegidas e ajudam o Governo do Estado a combater o novo Coronavírus (COVID-19). ',
        'start-online-service-v2': ' É muito importante responder todas as perguntas e completar o atendimento. Ao final, se for necessário, uma equipe de saúde estará pronta para conversar e cuidar de você.',
        'ask-patient-symptoms-v1': 'Você está com algum desses sintomas?   1. Falta de ar  2. Mal-estar intenso  3. Tosse incontrolável  4. Alargamento das narinas durante a respiração (para crianças)  5. Febre acima de 37.8°C persistente por mais de 48h  6. Confusão mental   7. Nenhum dos sintomas acima   Digite os números dos sintomas separados por vírgula (exemplo: 2, 5) ou digite 7 se não estiver com os sintomas descritos acima.',
        'ask-patient-symptons-v2': 'Você está com algum desses sintomas?  Selecione uma ou mais das opções abaixo.',
        'ask-patient-symptons-v3': 'Você está com algum desses outros sintomas?  1. Febre  2. Tosse  3. Nariz escorrendo  4. Dor de cabeça  5. Dor de garganta  6. Dores pelo corpo  7. Diarreia  8. Nenhum dos sintomas acima   Digite os números dos sintomas separados por vírgula (exemplo: 2, 5, 7) ou digite 8 se não estiver com os sintomas descritos acima.',
        'ask-patient-symptons-v4': 'Você está com algum desses outros sintomas? Selecione uma ou mais das opções abaixo.',
        'ask-patient-how-many-days-symptoms': 'Há quantos dias os sintomas começaram? Responda apenas com o número de dias (exemplo: 2).',
        'call-diagnostic-system': 'O sistema está verificando se você está com suspeita de covid.',
        'ask-share-location-v1': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no botão "Compartilhar minha localização" para compartilhar onde você está. Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, clique no botão "Não quero compartilhar minha localização".',
        'ask-share-location-v2': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no botão ""Compartilhar minha localização"" para compartilhar onde você está. Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, clique no botão ""Não quero compartilhar minha localização"".',
        'ask-share-location-v3': 'Vamos direcioná-lo(a) para nossa equipe de saúde. Clique no  ou no , selecione "Localização" e depois "Localização fixa". Ao compartilhar sua localização, você ajuda o Governo a te proteger e proteger os outros. Caso não deseje compartilhar, digite "Não".',
        'ask-share-location-v4': 'Clique no  ou no , selecione "Localização" e depois "Localização fixa" e compartilhe onde você está. Ao fazer isso, você ajuda o Governo a cuidar melhor de você. Caso não deseje compartilhar, digite "Não".',
        'show-info-about-share-location': 'Localização compartilhada com sucesso! ',
        'error-treatment-share-location': 'O compartilhamento da localização falhou.  Vamos tentar de outra forma.',
        'ask-postal-code': 'Informe o CEP da sua localidade, por favor. Exemplo: 57000-000 ou 57000000. Se não souber seu CEP, digite "Não".',
        'error-treatment-postal-code': 'CEP inválido! Por favor, me informe um CEP válido com o formato abaixo: Exemplo: 57000-000 ou 57000000',
        'ask-patient-info-state': 'Qual o seu estado? Exemplo: Ceará ou CE',
        'ask-patient-info-city': 'Qual a sua cidade?',
        'ask-patient-info-cpf': 'Você pode me informar seu CPF? Não precisa digitar os pontos ou traços ',
        'ask-patient-info-name': 'Qual seu nome completo?',
        'ask-patient-info-birthday': 'Qual a sua data de nascimento? Por favor, insira uma data no formato dd/mm/aaaa.',
        'error-treatment-patient-info-birthday': 'Data inválida! Por favor, digite uma data válida no formato dd/mm/aaaa.',
        'ask-patient-comorbidities-v1': 'Você tem algum desses problemas de saúde? 1. Diabetes 2. Pressão alta 3. Problemas nos rins 4. Problemas respiratórios 5. Problemas cardíacos 6. Deficiências imunológicas: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides 7. Nenhum dos problemas acima Digite os números dos sintomas separados por vírgula (exemplo: 2, 5) ou digite 7 se não estiver com os sintomas descritos acima.',
        'ask-patient-comorbidities-v2': 'Você tem algum desses problemas de saúde? Selecione uma ou mais das opções abaixo. *Considere deficiência imunológica: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides',
        'ask-patient-comorbidities-v3': 'Você tem algum desses problemas de saúde?    1. Diabetes  2. Pressão alta  3. Problemas nos rins  4. Problemas respiratórios  5. Problemas cardíacos  6. Deficiências imunológicas: portador de HIV, em tratamento para algum câncer, portador de doenças autoimunes (como lúpus, síndrome antifosfolípide, ...), realizou transplante ou usa corticoides  7. Nenhum dos problemas acima   Digite os números dos problemas separados por vírgula (exemplo: 2, 5) ou digite 7 se não tiver os problemas de saúde descritos acima.',
        'ask-patient-info-gender': 'Você é homem, mulher ou prefere não declarar?',
        'ask-patient-info-pregmant-woman': 'Você está grávida?',
        'ask-patient-info-phone-number': 'me informa o número do seu celular, por favor? Não esquece do DDD. Exemplo: (82) 90000-0000 ou 82912345678',
        'show-info-about-mental-health': 'você gostaria de tirar alguma dúvida sobre os temas a seguir? (Basta digitar o número, um por vez, para acessar a resposta) 1) Como cuidar da minha saúde mental durante a pandemia?  2) Como lidar com as crianças nesse período de isolamento? 3) Como lidar com os idosos nesse período? 4) Como apoiar alguém que está muito ansioso com o novo Coronavírus (COVID-19)? 5) Onde encontro serviços de saúde mental durante a pandemia? 6) Fale com a Equipe de Saúde Mental online 7) Para encerrar seu atendimento',
        'show-info-mental-health-option-01': 'Alimente-se bem e se exercite Cultive seus hobbies, pratique atividades artísticas Medite Fortaleça os laços com a família e os amigos Diminua o tempo gasto com notícias e busque fontes confiáveis Proteja-se e ajude o próximo Divulgue notícias positivas e de esperança Evite combater o estresse usando tabaco, álcool ou outras drogas Peça ajuda a alguém de sua confiança, se sentir que está sobrecarregado(a) emocionalmente Pense no que o(a) ajudou a lidar com os problemas no passado e o que pode fazer para se fortalecer.',
        'show-info-about-covid': 'você gostaria de tirar alguma dúvida sobre os temas a seguir? (digite um número de cada vez para ver a resposta) 1) O que é novo Coronavírus (COVID-19)?  2) Como faço para me proteger?  3) Quais são os principais sintomas?  4) Qual o tratamento?  5) Quais são os riscos?  6) O que fazer quando sentir os sintomas?   7) Mitos e verdades sobre a doença  8) Informações sobre o novo Coronavírus no Estado do Ceará  ou 9) Para encerrar seu atendimento. ',
        'show-info-about-new-corona-virus': 'Coronavírus é uma família de vírus que causam infecções respiratórias. Um novo Coronavírus foi descoberto em dezembro de 2019, na China. Ele causa a doença chamada de COVID-19.',
        'show-info-about-main-symptoms': 'Os principais sintomas da doença do novo Coronavírus são: ✓Falta de ar ✓Febre ✓Tosse seca',
        'show-info-about-the-treatments': 'Até o momento, não há vacina nem medicamento para prevenir ou tratar o novo Coronavírus (COVID-19). As pessoas afetadas devem receber cuidados gerais, e aquelas com doenças graves devem procurar atendimento de saúde.',
        'show-info-about-covid-in-ceara-v1': 'Para mais informações, acesse os canais oficiais da Secretaria da Saúde do Estado do Ceará (SESA), do Ministério da Saúde e da Organização Mundial da Saúde (OMS). ',
        'show-info-about-covid-in-ceara-v2': 'https://www.saude.ce.gov.br',
        'show-info-about-covid-in-ceara-v3': 'https://www.saude.gov.br',
        'show-info-about-covid-in-ceara-v4': 'https://coronavirus.ceara.gov.br',
        'show-info-about-covid-in-ceara-v5': 'Colabore! Não divulgue conteúdos que não tenham sido produzidos por fontes confiáveis e não espalhe fake news. ',
        'show-info-about-how-to-protect-from-virus': 'Consegui entender que você quer saber mais sobre proteção e prevenção. O que você gostaria de saber? 1. Quais as principais medidas de prevenção? 2. Devo usar máscara? 3. Como deve ser a minha máscara caseira? 4. Posso usar minha máscara caseira mais de uma vez? 5. Como lavar as mãos da maneira certa? 6. Como os profissionais de saúde podem se proteger do novo Coronavírus (COVID-19)?',
        'show-info-about-prevention-measure': 'undefined path',
        'show-info-about-wear-mask': 'undefined path',
        'show-info-about-homemade-mask-look-like': 'undefined path',
        'show-info-about-homemade-mask-use': '✔ Você pode usar sua máscara caseira mais de uma vez, mas tenha os seguintes cuidados: 1. Lave bem as mãos e tire a máscara sem tocar na parte da frente. 2. Deixe a máscara de molho por meia hora em água + água sanitária. 3. Enxágue e lave com água e sabão. Espere secar para usá-la de novo. ⚠ Atenção! Leve com você reservas de máscara, porque o tempo adequado de uso é de 2 horas.',
        'show-info-about-wash-hands-correctly': 'undefined path',
        'show-info-health-professional-protection': 'undefined path',
        'show-info-about-the-riscs': 'Consegui entender que você quer saber mais sobre riscos. O que você gostaria de saber? 1. Como essa doença se apresenta? 2. Como essa doença se espalha? 3. Essa doença é transmitida pelo ar? 4. Devo me preocupar com essa doença? 5. Quem é grupo de risco?',
        'show-info-about-disease-present-itself': '- Os sintomas demoram de 1 a 14 dias para aparecer. - A maioria das pessoas se recupera sem precisar de tratamento especial. - A doença pode se agravar em idosos ou naqueles que já tenham outros problemas de saúde (pressão alta, diabetes ou problemas no coração). - As crianças geralmente têm sintomas leves ou não apresentam sintomas.',
        'show-info-about-disease-spread': 'undefined path',
        'show-info-about-disease-transmitted-through-the-air': 'A doença não é transmitida pelo ar. Até agora, estudos indicam que o vírus é passado principalmente pelo contato com gotículas respiratórias.',
        'show-info-about-worried-about-this-disease': 'undefined path',
        'show-info-about-the-risk-group': 'Idosos e pessoas com outros problemas de saúde (diabetes, pressão alta e problemas de coração, por exemplo) são grupo de risco.',
        'show-info-about-when-feel-the-symptoms': 'Consegui entender que você quer saber mais sobre sintomas. O que você gostaria de saber? 1. Não me sinto bem. Devo procurar um médico? 2. Onde buscar atendimento? 3. Como é feito o diagnóstico do novo Coronavírus (COVID-19)? 4. Para quem o exame é indicado? 5. O que é isolamento domiciliar? 6. Quando é preciso fazer isolamento domiciliar? 7. O que é o distanciamento social? 8. Todo caso de COVID-19 precisa ser internado no hospital?',
        'show-info-about-should-see-the-doctor': 'undefined path',
        'show-info-about-where-to-get-care': 'undefined path',
        'show-info-about-the-diagnostic-of-the-covid': 'Atualmente, o diagnóstico é feito pela técnica swab, que é o uso de um bastão, parecido com um cotonete, para coletar materiais respiratórios (aspiração de vias aéreas ou coleta de secreções da boca e do nariz).',
        'show-info-about-who-is-the-exam-for': 'O exame é indicado para: ✓ Pessoas com sintomas classificados como Síndrome Respiratória Aguda Grave (SRAG)*✓ Profissionais da saúde com sintomas ✓ Pessoas vulneráveis com síndrome gripal (a partir de 60 anos e jovens com diabetes mellitus, pressão alta, problemas cardíacos, doença pulmonar crônica, câncer e gravidez de risco) * Síndrome Respiratória Aguda Grave (SRAG): febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo.',
        'show-info-about-home-isolation': 'undefined path',
        'show-info-about-when-home-isolation-is-necessary': 'undefined path',
        'show-info-about-social-distancing': 'undefined path',
        'show-info-about-the-cases-need-hospitalization': 'Nem todo caso de COVID-19 precisa ser internado. Somente pacientes classificados como Síndrome Gripal (SG)* e sinais de alarme (desconforto respiratório, cansaço, respiração rápida) ou Síndrome Respiratória Aguda Grave (SRAG)**. * Síndrome Gripal (SG):  febre acompanhada de tosse ou dor de garganta ou falta de ar ou nariz escorrendo. Em criança com menos de 2 anos: febre de início súbito + sintomas respiratórios (tosse, coriza e obstrução nasal). ** Síndrome Respiratória Aguda Grave (SRAG): indivíduo com febre acompanhada de tosse ou dor de garganta e que apresenta falta de ar.',
        'show-info-about-myths-and-truths': 'Consegui entender que você quer saber mais sobre mitos. O que você gostaria de saber? 1. Os animais de estimação podem espalhar o novo Coronavírus? 2. Quais as pessoas mais afetadas pelo novo Coronavírus? 3. Os antibióticos são eficazes na prevenção e no tratamento do novo Coronavírus? 4. Posso tomar algum remédio para prevenir ou tratar novo Coronavírus? 5. Quanto tempo o novo Coronavírus sobrevive em superfícies? 6. O novo Coronavírus pode matar? 7. Há restrições para comprar mercadorias vindas de países com casos confirmados da doença?',
        'show-info-about-pets-spread-covid': 'undefined path',
        'show-info-about-people-affected-by-covid': 'undefined path',
        'show-info-antibiotics-effects': 'Antibióticos *não* são eficazes para prevenir e tratar o novo Coronavírus. Antibióticos só funcionam contra bactérias. O novo Coronavírus (COVID-19) é um vírus',
        'show-info-about-medicine-for-covid': 'Ainda não existem remédios específicos recomendados para prevenir ou tratar o novo Coronavírus (COVID-19).',
        'show-info-about-covid-survive-in-surface': 'undefined path',
        'show-info-about-covid-kill': 'undefined path',
        'show-info-about-severity-of-covid': 'Em geral, a doença é leve, mas pode se agravar. Cerca de 1 em cada 5 pessoas infectadas necessita de cuidados hospitalares.',
        'show-info-about-the-restrictions-to-by': 'undefined path',
        'show-info-about-follow-program-v1': 'Faça parte do nosso Programa de Acompanhamento de Saúde, envie uma mensagem de Whatsapp com a palavra "Acompanhamento" para o número (85) 98439-0647.  Ou clique no link abaixo para ser direcionado para nosso perfil no Whatsapp: ',
        'show-info-about-follow-program-v2': 'https://bit.ly/2KitO8J',
        'show-info-about-follow-program-v3': 'Assim, mandaremos mensagens para saber como você está se sentindo e garantir que está recebendo a orientação certa.',
        'show-info-about-follow-program-v4': 'A Secretaria da Saúde do Estado tem um Programa de Acompanhamento de Saúde para você.  Mandaremos mensagens para saber como você está se sentindo e garantir que está recebendo a orientação certa.',
        'show-info-about-follow-program-v5': 'Pessoal, o governo lançou um serviço de atendimento pelo WhatsApp que pode nos ajudar.  Basta clicar nesse link para ser atendido(a) online por um profissional de saúde!',
        'show-info-about-folow-program-v6': 'https://bit.ly/ceara_plantaocoronavirus',
        'show-info-others': 'Este é um canal para te orientar sobre o novo Coronavírus (COVID-19).     Para denunciar descumprimento a determinações do Governo no enfrentamento ao novo Coronavírus, ligue 190.      Para outras informações, fale com a Central de Atendimento da Ouvidoria pelo 155.',
        'ask-user-grade-v1': 'Certo! Antes de encerrar peço 1 minutinho para que você avalie o meu atendimento (digite um número de 1 a 10), sendo: 1. Odiei.  >>> 10. Amei! ',
        'ask-user-grade-v2': 'Por favor, antes de desconectar, avalie esse serviço. Dê uma nota de 1 a 10, sendo:   1.Ruim  >>> 10. Muito bom! ',
        'ask-user-check-v1': 'Suas dúvidas foram respondidas?',
        'ask-user-check-v2': 'Quer fazer outra pergunta?',
        'ask-user-check-v3': ' você gostaria de tirar alguma dúvida sobre os temas a seguir?',
        'finish-service': 'Seu atendimento foi finalizado.',
        'finish-dialog-final-v1': 'Obrigado pelo seu contato! Juntos venceremos essa luta contra o novo Coronavírus! #FiqueEmCasa    Até logo!',
        'finish-dialog-final-v2': 'Seja um(a) aliado(a) nessa luta, compartilhe esse serviço com pessoas e grupos que podem precisar da nossa ajuda.   Juntos venceremos essa luta contra o novo Coronavírus! #FiqueEmCasa    Até logo!',
        'health-agent-takes-control': 'Olá, obrigado pelas informações! Elas são muito importantes para que nós consigamos atender a todos nesse momento difícil. Nossa equipe de saúde está aqui para te ajudar! Como posso te ajudar?',
        'show-info-calling-health-profitional': 'Estamos acionando um profissional de saúde para atender e cuidar de você.  Por favor, complete as informações a seguir para agilizarmos o seu atendimento.',
        'show-info-diagnostic-positive-v1': 'De acordo com suas respostas e com o protocolo do Ministério da Saúde, *você pode estar com o novo Coronavírus (COVID-19).*  ',
        'show-info-diagnostic-positive-v2': 'De acordo com suas respostas e com o protocolo do Ministério da Saúde, *você possui sintomas leves que podem estar relacionados ao novo Coronavírus (COVID-19).*  ',
        'show-info-diagnostic-negative': 'undefined path',
        'call-health-agent-v1': 'Obrigado por se cadastrar. Permaneça com a conversa aberta. Em alguns minutos iremos atendê-lo(a). ',
        'call-health-agent-v2': 'Obrigado por se cadastrar. Permaneça com a conversa aberta. Em alguns minutos iremos atendê-lo(a).',
        'call-health-agent-v3': 'Vou te encaminhar para nossa equipe de saúde. ',
        'call-health-agent-v4': 'Vamos direcioná-lo(a) para nossa Equipe de Saúde Mental. ',
        'call-health-agent-v5': 'Continue com a nossa conversa aberta. A Equipe de Saúde Mental vai atendê-lo(a) em alguns minutos. ',
        'show-initial-message-follow-program-v1': 'Queremos estar mais próximos de você através do nosso Programa de Acompanhamento de Saúde. ',
        'show-initial-message-follow-program-v2': 'Estamos de volta com nosso Programa de Acompanhamento de Saúde para estar mais próximos de você. ',
        'start-dialog-follow-program-v1': 'Agora, faremos algumas perguntas sobre como você está se sentindo hoje.',
        'star-dialog-follow-program-v2': 'As informações compartilhadas aqui são protegidas e ajudam o Governo do Estado no combate ao novo Coronavírus (COVID-19).'
    }

    #print('-------', phrase)
    #action = [k for k, v in actions_map.items() if v == phrase]
    action = [key for key, val in actions_map.items() if val in phrase]

    if action:
      action = action[0]


    return action


# **Análise dos diálogos do dataframe**

In [ ]:
# extração das ações a cada interação do chatbot

for i in range(id_dialogos.size):
  dialog = df_by_id_chat.loc[id_dialogos[i]]
  if (isinstance(dialog,pd.DataFrame)):
    print('Caminho do diálogo', i, '\n')
    for index, row in dialog.iterrows():
      if (row['ori'] == 'bot'):
        print(find_actions(row.txt), '\n')
    print('\n')


In [ ]:
# extração dos predicados do usuário a cada interação do usuário

# for i in range(1):
#   initial_predicates = predicates()
#   int_predicates = initial_predicates[0]
#   ent_predicates = initial_predicates[1]
#   dialog = df_by_id_chat.loc[id_dialogos[i]]
#   print('Caminho do diálogo', i, '\n')
#   for index, row in dialog.iterrows():
#     if (row['ori'] == 'patient'):
#       results = predicates_extraction(row.txt, int_predicates, ent_predicates)
#       int_predicates = results[0]
#       ent_predicates = results[1]
#       print('Predicados do usuário: ', results[2])
#   print('\n')